In [1]:
%matplotlib inline

import xrft
import math
import time
import scipy.io
import numpy as np
import xarray as xr
import netCDF4 as nc
import cmocean as cm
import matplotlib
from scipy import signal
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.gridspec as gridspec

from gsw import f
from matplotlib.colors import LogNorm
from scipy.fftpack import fft2, fftn, fftshift

import dask

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_delta(lat):

    # compute the distance in km at this lon and lat
    delta_lon = 2 * np.pi * (6371 * np.cos(lat*np.pi/180)) / 360
    delta_lat = 1 * np.pi * 6371 / 180
    
    return delta_lon, delta_lat

In [3]:
# select a domain, 10 deg by 10 deg
topog = xr.open_dataset('/g/data/nm03/lxy581/synbath/SYNBATH.nc')
topog_sample = topog.z[240*135:240*145,240*145:240*155]

topog_sample -= topog_sample.mean(skipna=True)

ydim, xdim = topog_sample.dims
nx = topog_sample[xdim].size
ny = topog_sample[ydim].size

lon_sample = topog_sample.lon
lat_sample = topog_sample.lat

yind = 240*140-1
xind = 240*150-1
print('Location for the centre point: \nLon: %.1f W;' % np.abs(topog.lon[xind].values),'\nLat: %.1f N.' % topog.lat[yind].values)

Location for the centre point: 
Lon: 30.0 W; 
Lat: 50.0 N.


In [4]:
# fit the filtered 2D topography with a polynomial sloping surface
def get_plane_fit(topog_tile, lon_tile, lat_tile):
    LON, LAT = np.meshgrid(lon_tile, lat_tile, copy=False)
    X = LON.flatten()
    Y = LAT.flatten()
    A = np.array([np.ones_like(X), X, Y, X*Y]).T
    B = np.array(topog_tile).flatten()
    coeff, r, rank, s = np.linalg.lstsq(A, B)
    topog_fit = coeff[0] + coeff[1]*LON + coeff[2]*LAT + coeff[3]*LON*LAT
    return topog_fit

In [5]:
print('Mean before changes: ', topog_sample.mean(skipna=True).values)

topog_fit = get_plane_fit(topog_sample, lon_sample, lat_sample)

# instead of removing the mean, subtracting a plane fit
topog_diff = topog_sample - topog_fit
print('Mean after changes: ', topog_diff.mean(skipna=True).values)

var_JSL = np.nansum(topog_diff ** 2)
print('variance of deviation from plane fit (h_JSL) = %3.2e \n' % var_JSL)

Mean before changes:  0.002649075
Mean after changes:  -6.998772833968461e-10
variance of deviation from plane fit (h_JSL) = 1.33e+12 



In [6]:
# varlist = ['topog_sample','topog_fit']
# title   = ['Sampled topography (demeaned)', 'Plane fit']
# levels  = np.arange(-3000,3000+100,100)

# fig = plt.figure(figsize=(12, 6))
# axs = []

# gridsubs = gridspec.GridSpec(1,2)
# for gs in gridsubs:
#     axs.append(plt.subplot(gs))

# for I, (ax, var) in enumerate(zip(axs, varlist)):

#     p = ax.contourf(globals()[var], levels=levels, cmap='bwr', vmin=-3000, vmax=3000, extend="both") 

#     ax.set_title(title[I],fontsize=16)
#     ax.set_xticks(np.arange(0,240*10+240*5,240*5))
#     ax.set_yticks(np.arange(0,240*10+240*5,240*5))
    
#     ax.set_xticklabels(['35$^{\circ}$W','30$^{\circ}$W','25$^{\circ}$W'],fontsize=14)
#     ax.set_yticklabels(['45$^{\circ}$N','50$^{\circ}$N','55$^{\circ}$N'],fontsize=14)
    
#     if I==0:
#         ax.set_position([0.1,0.1,0.35,0.75])
#     if I==1:
#         ax.set_position([0.52,0.1,0.35,0.75])

#     tick_locs = np.arange(-3000,3000+1000,1000)
#     tick_labels = np.array(['-3','-2','-1','0','1','2','3'])
    
#     if I==1:
#         ax_cb = plt.axes([0.89, 0.1, 0.01, 0.75])  
#         cb = plt.colorbar(p, cax=ax_cb, orientation='vertical', extend='both')
#         cb.ax.set_ylabel('Depth (km)',fontsize=16,rotation=270,labelpad=25);
#         cb.ax.tick_params(labelsize=16)
#         cb.locator   = matplotlib.ticker.FixedLocator(tick_locs)
#         cb.formatter = matplotlib.ticker.FixedFormatter(tick_labels)

In [7]:
var_topog_tile = ['topog_tile_bl','topog_tile_br',
                  'topog_tile_tl','topog_tile_tr']

# bottom left 
topog_tile_bl = topog_sample[0:int(ny/2),0:int(nx/2)]
lon_bl = lon_sample[0:int(nx/2)]
lat_bl = lat_sample[0:int(ny/2)]
topog_fit_bl = get_plane_fit(topog_tile_bl, lon_bl, lat_bl)
topog_diff_bl = topog_tile_bl - topog_fit_bl
var_JSL_bl = np.nansum(topog_diff_bl ** 2)
print('variance of deviation from plane fit (h_JSL) = %.2f \n' % var_JSL_bl)

# bottom right
topog_tile_br = topog_sample[0:int(ny/2),int(nx/2):]
lon_br = lon_sample[int(nx/2):]
lat_br = lat_sample[0:int(ny/2)]
topog_fit_br = get_plane_fit(topog_tile_br, lon_br, lat_br)
topog_diff_br = topog_tile_br - topog_fit_br
var_JSL_br = np.nansum(topog_diff_br ** 2)
print('variance of deviation from plane fit (h_JSL) = %.2f \n' % var_JSL_br)

# top left 
topog_tile_tl = topog_sample[int(ny/2):,0:int(nx/2)]
lon_tl = lon_sample[0:int(nx/2)]
lat_tl = lat_sample[int(ny/2):]
topog_fit_tl = get_plane_fit(topog_tile_tl, lon_tl, lat_tl)
topog_diff_tl = topog_tile_tl - topog_fit_tl
var_JSL_tl = np.nansum(topog_diff_tl ** 2)
print('variance of deviation from plane fit (h_JSL) = %.2f \n' % var_JSL_tl)

# top right
topog_tile_tr = topog_sample[int(ny/2):,int(nx/2):]
lon_tr = lon_sample[int(nx/2):]
lat_tr = lat_sample[int(ny/2):]
topog_fit_tr = get_plane_fit(topog_tile_tr, lon_tr, lat_tr)
topog_diff_tr = topog_tile_tr - topog_fit_tr
var_JSL_tr = np.nansum(topog_diff_tr ** 2)
print('variance of deviation from plane fit (h_JSL) = %.2f \n' % var_JSL_tr)

print('Totoal variance = %3.2e' % (var_JSL_bl + var_JSL_br + var_JSL_tl + var_JSL_tr))

variance of deviation from plane fit (h_JSL) = 133200639890.12 

variance of deviation from plane fit (h_JSL) = 337362542037.19 

variance of deviation from plane fit (h_JSL) = 218504433856.46 

variance of deviation from plane fit (h_JSL) = 182461099060.02 

Totoal variance = 8.72e+11


### Reconstruct the sampled topography from plane fit removed tiles

In [8]:
topog_sample_re = np.zeros((ny,nx))

topog_sample_re[0:int(ny/2),0:int(nx/2)] = topog_diff_bl
topog_sample_re[0:int(ny/2),int(nx/2):] = topog_diff_br 
topog_sample_re[int(ny/2):,0:int(nx/2)] = topog_diff_tl 
topog_sample_re[int(ny/2):,int(nx/2):] = topog_diff_tr 

In [11]:
topog_fit_re = get_plane_fit(topog_sample_re, lon_sample, lat_sample)

# instead of removing the mean, subtracting a plane fit
topog_diff_re = topog_sample_re - topog_fit_re

var_JSL_re_samp = np.nansum(topog_sample_re ** 2)
print('variance of deviation from plane fit (reconstructed) = %3.2e \n' % var_JSL_re_samp)

var_JSL_re_diff = np.nansum(topog_diff_re ** 2)
print('variance of deviation from plane fit (reconstructed, plane fit removed) = %3.2e \n' % var_JSL_re_diff)

var_JSL = np.nansum(topog_diff ** 2)
print('variance of deviation from plane fit (orginial, plane fit removed) = %3.2e \n' % var_JSL)

variance of deviation from plane fit (reconstructed) = 8.72e+11 

variance of deviation from plane fit (reconstructed, plane fit removed) = 8.72e+11 

variance of deviation from plane fit (orginial, plane fit removed) = 1.33e+12 

